In [1]:
sentence = "helo szia jános+"
[ord(x) for x in sentence] # getting unicode representation

[104, 101, 108, 111, 32, 115, 122, 105, 97, 32, 106, 225, 110, 111, 115, 43]

In [2]:
list(sentence.encode('utf-8')) # utf-8 encoding

[104,
 101,
 108,
 111,
 32,
 115,
 122,
 105,
 97,
 32,
 106,
 195,
 161,
 110,
 111,
 115,
 43]

# Byte Pair Encoding (BPE)

In [3]:
text = """size_average (bool, optional) – Deprecated (see reduction). By default, the losses are averaged over each loss element in the batch. Note that for some losses, there are multiple elements per sample. If the field size_average is set to False, the losses are instead summed for each minibatch. Ignored when reduce is False. Default: True reduce (bool, optional) – Deprecated (see reduction). By default, the losses are averaged or summed over observations for each minibatch depending on size_average. When reduce is False, returns a loss per batch element instead and ignores size_average. Default: True reduction (str, optional) – Specifies the"""
tokens = text.encode('utf-8')
tokens = list(map(int, tokens))
print(tokens)

[115, 105, 122, 101, 95, 97, 118, 101, 114, 97, 103, 101, 32, 40, 98, 111, 111, 108, 44, 32, 111, 112, 116, 105, 111, 110, 97, 108, 41, 32, 226, 128, 147, 32, 68, 101, 112, 114, 101, 99, 97, 116, 101, 100, 32, 40, 115, 101, 101, 32, 114, 101, 100, 117, 99, 116, 105, 111, 110, 41, 46, 32, 66, 121, 32, 100, 101, 102, 97, 117, 108, 116, 44, 32, 116, 104, 101, 32, 108, 111, 115, 115, 101, 115, 32, 97, 114, 101, 32, 97, 118, 101, 114, 97, 103, 101, 100, 32, 111, 118, 101, 114, 32, 101, 97, 99, 104, 32, 108, 111, 115, 115, 32, 101, 108, 101, 109, 101, 110, 116, 32, 105, 110, 32, 116, 104, 101, 32, 98, 97, 116, 99, 104, 46, 32, 78, 111, 116, 101, 32, 116, 104, 97, 116, 32, 102, 111, 114, 32, 115, 111, 109, 101, 32, 108, 111, 115, 115, 101, 115, 44, 32, 116, 104, 101, 114, 101, 32, 97, 114, 101, 32, 109, 117, 108, 116, 105, 112, 108, 101, 32, 101, 108, 101, 109, 101, 110, 116, 115, 32, 112, 101, 114, 32, 115, 97, 109, 112, 108, 101, 46, 32, 73, 102, 32, 116, 104, 101, 32, 102, 105, 101, 108, 1

In [4]:
def get_pairs(tokens):
    counts= {}
    for pair in zip(tokens, tokens[1:]): # loop through every two bytes
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_pairs(tokens)
sorted(((v,k) for k, v in stats.items()), reverse=True) # sorting list

[(22, (101, 32)),
 (16, (114, 101)),
 (13, (115, 32)),
 (13, (101, 100)),
 (12, (101, 114)),
 (11, (115, 101)),
 (11, (100, 32)),
 (9, (104, 101)),
 (9, (32, 116)),
 (8, (118, 101)),
 (8, (116, 105)),
 (8, (116, 104)),
 (8, (114, 32)),
 (8, (111, 110)),
 (8, (97, 116)),
 (8, (46, 32)),
 (8, (44, 32)),
 (8, (32, 115)),
 (8, (32, 111)),
 (8, (32, 97)),
 (7, (105, 111)),
 (7, (99, 104)),
 (7, (32, 114)),
 (7, (32, 105)),
 (6, (117, 99)),
 (6, (115, 115)),
 (6, (114, 97)),
 (6, (111, 115)),
 (6, (111, 114)),
 (6, (109, 101)),
 (6, (108, 111)),
 (6, (105, 110)),
 (6, (103, 101)),
 (6, (101, 115)),
 (6, (101, 110)),
 (6, (100, 117)),
 (6, (97, 118)),
 (6, (97, 108)),
 (6, (97, 103)),
 (6, (32, 108)),
 (6, (32, 101)),
 (5, (117, 108)),
 (5, (116, 101)),
 (5, (110, 32)),
 (5, (108, 116)),
 (5, (108, 101)),
 (5, (104, 32)),
 (5, (101, 97)),
 (5, (32, 40)),
 (4, (122, 101)),
 (4, (116, 99)),
 (4, (116, 32)),
 (4, (115, 105)),
 (4, (112, 101)),
 (4, (110, 115)),
 (4, (105, 122)),
 (4, (102, 97)),

In [5]:
chr(101), chr(32)

('e', ' ')

In [6]:
top_pair = max(stats, key=stats.get)
top_pair

(101, 32)

In [7]:
def merge(ids, pair, idx): #idx = replaced with
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]: # not the last element
            newids.append(idx) # replacing with idx, the replacement
            i += 2 # skip entire pair
        else:
            newids.append(ids[i]) # copy over original element
            i += 1
    return newids

merge([1, 2, 3, 3, 4, 5, 6], (3, 3), 69)

[1, 2, 69, 4, 5, 6]

In [8]:
tokens_replaced = merge(tokens, top_pair, 256) # 255 chars by default, so new index should start after
print(f'Original length: {len(tokens)}, replaced by top pair: {len(tokens_replaced)}')
tokens_replaced

Original length: 651, replaced by top pair: 629


[115,
 105,
 122,
 101,
 95,
 97,
 118,
 101,
 114,
 97,
 103,
 256,
 40,
 98,
 111,
 111,
 108,
 44,
 32,
 111,
 112,
 116,
 105,
 111,
 110,
 97,
 108,
 41,
 32,
 226,
 128,
 147,
 32,
 68,
 101,
 112,
 114,
 101,
 99,
 97,
 116,
 101,
 100,
 32,
 40,
 115,
 101,
 256,
 114,
 101,
 100,
 117,
 99,
 116,
 105,
 111,
 110,
 41,
 46,
 32,
 66,
 121,
 32,
 100,
 101,
 102,
 97,
 117,
 108,
 116,
 44,
 32,
 116,
 104,
 256,
 108,
 111,
 115,
 115,
 101,
 115,
 32,
 97,
 114,
 256,
 97,
 118,
 101,
 114,
 97,
 103,
 101,
 100,
 32,
 111,
 118,
 101,
 114,
 32,
 101,
 97,
 99,
 104,
 32,
 108,
 111,
 115,
 115,
 32,
 101,
 108,
 101,
 109,
 101,
 110,
 116,
 32,
 105,
 110,
 32,
 116,
 104,
 256,
 98,
 97,
 116,
 99,
 104,
 46,
 32,
 78,
 111,
 116,
 256,
 116,
 104,
 97,
 116,
 32,
 102,
 111,
 114,
 32,
 115,
 111,
 109,
 256,
 108,
 111,
 115,
 115,
 101,
 115,
 44,
 32,
 116,
 104,
 101,
 114,
 256,
 97,
 114,
 256,
 109,
 117,
 108,
 116,
 105,
 112,
 108,
 256,
 101,
 108,
 101,
 109,

In [9]:
VOCAB_SIZE = 276 # 20 merges ( 276 - 256 = 20)
num_merges = VOCAB_SIZE - 256
ids = list(tokens) # creating copy of `tokens` list

merges = {} # (child, child -> new token)
for i in range(num_merges):
    stats = get_pairs(ids)
    pair = max(stats, key=stats.get)

    idx = 256 + i # new token integer
    print(f'merging pair: {pair} into new token: {idx}')
    ids = merge(ids, pair, idx)
    merges[pair] = idx # store merge

merging pair: (101, 32) into new token: 256
merging pair: (101, 100) into new token: 257
merging pair: (115, 32) into new token: 258
merging pair: (101, 114) into new token: 259
merging pair: (115, 101) into new token: 260
merging pair: (118, 259) into new token: 261
merging pair: (44, 32) into new token: 262
merging pair: (116, 105) into new token: 263
merging pair: (111, 110) into new token: 264
merging pair: (46, 32) into new token: 265
merging pair: (116, 104) into new token: 266
merging pair: (263, 264) into new token: 267
merging pair: (97, 116) into new token: 268
merging pair: (257, 32) into new token: 269
merging pair: (99, 104) into new token: 270
merging pair: (97, 261) into new token: 271
merging pair: (271, 97) into new token: 272
merging pair: (272, 103) into new token: 273
merging pair: (97, 108) into new token: 274
merging pair: (114, 257) into new token: 275


In [10]:
print(f'tokens: {len(tokens)}')
print(f'ids: {len(ids)}')
print(f'compression: {len(tokens) / len(ids):.2f}x')

tokens: 651
ids: 475
compression: 1.37x


In [11]:
# decoding
vocab = {idx: bytes([idx]) for idx in range(256)} # tokenid - bytes object for token mapping
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1] # addition of two bytes

def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode('utf-8', errors='replace') # decoding bytes to string, errors occour for incorrect utf-8 sequences
    return text

In [14]:
# encoding
def encode(text):
    tokens = list(text.encode('utf-8')) # starting tokens (raw bytes)
    while len(tokens) >= 2:
        stats = get_pairs(tokens) # count occourence
        pair = min(stats, key=lambda x: merges.get(x, float('inf'))) # getting the pair's index, if it cannot be merged the value will be "inf"
        if pair not in merges:
            break # if nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode('helo sziasztok and'))

[104, 101, 108, 111, 32, 115, 122, 105, 97, 115, 122, 116, 111, 107, 32, 97, 110, 100]


In [15]:
valtext = """In PyTorch, the learnable parameters (i.e. weights and biases) of an torch.nn.Module model are contained in the model’s parameters (accessed with model.parameters()). A state_dict is simply a Python dictionary object that maps each layer to its parameter tensor. Note that only layers with learnable parameters (convolutional layers, linear layers, etc.) and registered buffers (batchnorm’s running_mean) have entries in the model’s state_dict. Optimizer objects (torch.optim) also have a state_dict, which contains information about the optimizer’s state, as well as the hyperparameters used."""
valtext_c = decode(encode(valtext))
print(valtext == valtext_c)

True


In [23]:
# force splits using regex
import regex as re
pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""") # GPT's pat

print(re.findall(pat, "encoding123 text've and's      AND'S!!??   "))

['encoding', '123', ' text', "'ve", ' and', "'s", '     ', ' AND', "'", 'S', '!!??', '   ']


## using tiktoken

In [24]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
print(enc.encode('    hello world!!!'))

enc = tiktoken.get_encoding('cl100k_base')
print(enc.encode('    hello world!!!'))

[220, 220, 220, 23748, 995, 10185]
[262, 24748, 1917, 12340]


## using sentencepiece

In [25]:
import sentencepiece as spm

In [26]:
with open("example.txt", "w", encoding="utf-8") as f:
  f.write("SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing.")

In [27]:
import os

# llama2 settings
options = dict(
  input="example.txt",
  input_format="text",
  model_prefix="tok400", # output filename prefix
  # algorithm spec
  # BPE alg
  model_type="bpe",
  vocab_size=400,
  # normalization
  normalization_rule_name="identity", # ew, turn off normalization
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995, # if occourence is below, it wont be in our vocab
  byte_fallback=True, # if unknown, fall back to bytes
  # merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True, # e.g. world is treated the same in `world` and `hello world`
  allow_whitespace_only_pieces=True,
  # special tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1, # the others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
  # systems
  num_threads=os.cpu_count(), # use ~all system resources
)


In [28]:
spm.SentencePieceTrainer.train(**options)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: example.txt
  input_format: text
  model_prefix: tok400
  model_type: BPE
  vocab_size: 400
  self_test_sample_size: 0
  character_coverage: 0.99995
  input_sentence_size: 200000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 8
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  d

In [29]:
sp = spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [31]:
ids = sp.encode("hello 안녕하세요")
print(ids)

[362, 378, 361, 372, 358, 362, 239, 152, 139, 238, 136, 152, 240, 152, 155, 239, 135, 187, 239, 157, 151]


In [32]:
print([sp.id_to_piece(idx) for idx in ids])

['▁', 'h', 'e', 'l', 'lo', '▁', '<0xEC>', '<0x95>', '<0x88>', '<0xEB>', '<0x85>', '<0x95>', '<0xED>', '<0x95>', '<0x98>', '<0xEC>', '<0x84>', '<0xB8>', '<0xEC>', '<0x9A>', '<0x94>']
